# Tesis Demo

Se importan las librerias respectivas a **pyspark**.

In [1]:
from pyspark import SparkContext
from pyspark.sql import HiveContext

sc = SparkContext('local[*]')
sqlContext = HiveContext(sc)

El siguiente bloque de codigo importa el modulo que integra las funciones desarrolladas:

In [2]:
module_path = "/home/jovyan/work/tools.py"
sc.addPyFile(module_path)
import tools

El siguiente bloque ejecuta una función del modulo que se acaba de cargar, verificando su correcto funcionamiento el resultado debe ser **0.9**.

In [3]:
tools.lexical_diversity("Hola Mundo")

0.9

En el siguiente bloque debe indicar las carpetas respectivas a tweets **SPAM** y **NO SPAM** para el entrenamiento del clasificador de tweets:

In [4]:
spamFile = "/home/jovyan/work/datos/spam"
noSpamFile = "/home/jovyan/work/datos/no_spam"
juez_spam = tools.entrenar_spam(sc, sqlContext, spamFile, noSpamFile, 3, 2)

En el siguiente bloque se entrenará el clasificador de timelines, para lo cual se deben especificar las direcciones respectivas a las carpetas que contienen los timelines de **humanos, ciborgs y bots**:

In [5]:
humanos = '/home/jovyan/work/datos/datos_cat/humano'
ciborgs = '/home/jovyan/work/datos/datos_cat/ciborg'
bots =  '/home/jovyan/work/datos/datos_cat/bot'
juez_usuarios = tools.entrenar_juez(sc, sqlContext, juez_spam, humanos, ciborgs, bots)

Obteniendo las caracteristicas de los usuarios:

In [8]:
timelinesInput = '/home/jovyan/work/datos/datos/*'
features = tools.timeline_features(juez_spam, tools.cargar_datos(sc, sqlContext, timelinesInput)).cache()
features.first()

En el siguiente bloque de codigo se realizará la evaluación de un usuario para categorizarlo:
* Humano = 0
* Bot = 1
* Ciborg = 2

In [12]:
resultado = tools.evaluar(sc, sqlContext, juez_spam, juez_usuarios, timelinesInput)

resultado.collect()

[{'ano_registro': 2010,
  'avg_diversidad': 0.9724475969798728,
  'avg_diversidad_lex': 0.38475247659424383,
  'avg_hashtags': 0.01706827309236948,
  'avg_long_tweets': 74.6004016064257,
  'avg_palabras': 12.89558232931727,
  'avg_spam': 0.025100401606425703,
  'con_descripcion': 1,
  'con_geo_activo': 1,
  'con_imagen_default': 0,
  'con_imagen_fondo': 1,
  'con_perfil_verificado': 0,
  'createdAt': datetime.datetime(2016, 7, 27, 22, 40, 4, 214881),
  'diversidad_url': 0,
  'entropia': -0.15307207909875986,
  'followers_ratio': 0.7169398907103826,
  'hora_0': 0.04618473895582329,
  'hora_1': 0.04618473895582329,
  'hora_10': 0.001004016064257028,
  'hora_11': 0.006024096385542169,
  'hora_12': 0.0050200803212851405,
  'hora_13': 0.029116465863453816,
  'hora_14': 0.024096385542168676,
  'hora_15': 0.04819277108433735,
  'hora_16': 0.05321285140562249,
  'hora_17': 0.05823293172690763,
  'hora_18': 0.03413654618473896,
  'hora_19': 0.03413654618473896,
  'hora_2': 0.06224899598393574,
